In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/janita/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
cosmetics_data = pd.read_csv('content_main.csv')

In [5]:
cosmetics_data.isna().sum()

product_name       0
category           0
ingredients        0
price              0
brand             72
rating          2070
product_id         0
description     2070
Combination     1453
Dry             1453
Normal          1453
Oily            1453
Sensitive       1453
dtype: int64

In [6]:
cosmetics_data.head()

,product_name,category,ingredients,price,brand,rating,product_id,description,Combination,Dry,Normal,Oily,Sensitive
0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",5.2,The Ordinary,4.55,315,Effectively hydrate skin with The Ordinary Nat...,NaN,NaN,NaN,NaN,NaN
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",13.0,CeraVe,4.27,316,Protect your skin with,NaN,NaN,NaN,NaN,NaN
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",6.2,The Ordinary,4.70,317,Recharge and rehydrate your complexion with the,NaN,NaN,NaN,NaN,NaN
3,AMELIORATE Transforming Body Lotion 200ml,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",22.5,AMELIORATE,NaN,318,NaN,NaN,NaN,NaN,NaN,NaN
4,CeraVe Moisturising Cream 454g,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",16.0,CeraVe,4.71,319,Envelope your skin in the replenishing powers ...,NaN,NaN,NaN,NaN,NaN


In [7]:
cosmetics_data['rating'] = cosmetics_data['rating'].fillna(0)
cosmetics_data['description'] = cosmetics_data['description'].fillna('')
cosmetics_data['brand'] = cosmetics_data['brand'].fillna('')

In [9]:
cosmetics_data.drop(columns = ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive'], inplace = True)

### Non-personalised recommmendation

In [10]:
import pandas as pd

# Compute the average rating for each product
product_ratings = cosmetics_data.groupby('product_id')['rating'].mean()

# Sort the products based on the average rating
top_products = product_ratings.sort_values(ascending=False)

# Convert the Series object to a DataFrame
top_products_df = top_products.reset_index(name='avg_rating')

# Print the top 10 recommended products
print(top_products_df.head(10))

   product_id  avg_rating
0        1331         5.0
1        1393         5.0
2         720         5.0
3         724         5.0
4        1394         5.0
5        1093         5.0
6        1091         5.0
7         728         5.0
8         730         5.0
9        1084         5.0


### Content-Based Recommendation - based on category, description and ingredients

In [11]:
# Create a tf-idf vectorizer object
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [12]:

# Apply tf-idf vectorization to the 'type' and 'ingredients' columns
cosmetics_tfidf1 = tfidf_vectorizer.fit_transform(cosmetics_data['category'] + ' ' + cosmetics_data['ingredients']+ cosmetics_data['description'])


In [13]:
cosine_similarities1 = cosine_similarity(cosmetics_tfidf1)

In [14]:


def get_top_similar_products1(product_id, N=10, min_similarity=0):
    # Get the index of the product
    product_index = cosmetics_data[cosmetics_data['product_id'] == product_id].index[0]
    
    # Get the cosine similarity scores for the product
    cosine_scores = cosine_similarities1[product_index]
    # Sort the cosine similarity scores in descending order and get the top N similar products
    similar_product_indices = cosine_scores.argsort()[::-1][:N+1]
    similar_product_indices = similar_product_indices[similar_product_indices != product_index]
    # Filter out products with similarity score less than min_similarity
    similar_products = cosmetics_data.loc[similar_product_indices].copy()
    similar_products['similarity_score'] = cosine_scores[similar_product_indices]
    similar_products = similar_products[similar_products['similarity_score'] >= min_similarity]
    # Sort the similar products based on rating
    similar_products['rating'] = pd.to_numeric(similar_products['rating'], errors='coerce')
    similar_products.sort_values(by='rating', ascending=False, inplace=True)
    # Return the top N similar products
    return similar_products.iloc[:N]

# Example usage
get_top_similar_products1(322)

,product_name,category,ingredients,price,brand,rating,product_id,description,similarity_score
579,Peter Thomas Roth Peptide 21 Amino Acid Exfoli...,Peel,"['butylene glycol', 'sodium lactate', 'phytic ...",41.6,Peter Thomas Roth,5.00,894,Enjoy skin that looks smoother and more even w...,0.337983
355,Zelens PROVITAMIN D Fortifying Facial Mist 50ml,Mist,"['propanediol', 'glycerin', 'polyglyceryl-3 pc...",55.0,Zelens,5.00,670,Refresh and rehydrate thirsty skin with a few ...,0.316930
116,The Ordinary Buffet Supersize Serum 60ml,Serum,"['glycerin', 'lactococcus ferment lysate', 'ac...",24.4,The Ordinary,4.77,431,Rejuvenate your complexion with,0.507270
327,Zelens Z Balance Prebiotic and Probiotic Facia...,Mist,"['butylene glycol', 'sodium lactate', 'lactoba...",48.0,Zelens,4.60,642,Refresh your complexion with the,0.368792
0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",5.2,The Ordinary,4.55,315,Effectively hydrate skin with The Ordinary Nat...,0.569185
145,The Ordinary Amino Acids + B5,Serum,"['propanediol', 'betaine', 'sodium pca', 'pant...",5.9,The Ordinary,4.41,460,Hydrate and strengthen skin with,0.547942
683,Sarah Chapman Lash Boosting Eye Cleanse 70ml,Eye Care,"['glycerin', 'olea europaea fruit oil', 'rosa ...",33.0,Sarah Chapman Skinesis,2.00,998,Revitalise tired eyes and effortlessly remove ...,0.344271
367,ARK Skincare Hydrating Beauty Mist 150ml,Mist,"['glycerin', 'polysorbate-20', 'phenoxyethanol...",25.0,,0.00,682,,0.430588
2129,FAB Skin Lab Retinol Serum 0.25% Pure Concentrate,Treatment,"Water, Polysorbate 80, Caprylic/Capric Triglyc...",58.0,FIRST AID BEAUTY,0.00,2130,,0.324834
1253,FAB Skin Lab Retinol Serum 0.25% Pure Concentrate,Treatment,"['polysorbate', 'caprylic', 'capric', 'triglyc...",58.0,FIRST AID BEAUTY,0.00,115,,0.302402


### Content-Based Recommendaton - based on category, ingredients, brand and description

In [16]:

# Apply tf-idf vectorization to the 'type' and 'ingredients' columns
cosmetics_tfidf2 = tfidf_vectorizer.fit_transform(cosmetics_data['category'] + ' ' + cosmetics_data['ingredients']+' '+ cosmetics_data['description']+ ' '+cosmetics_data['brand'])


In [17]:
cosine_similarities2 = cosine_similarity(cosmetics_tfidf2)

In [18]:


def get_top_similar_products2(product_id, N=10, min_similarity=0):
    # Get the index of the product
    product_index = cosmetics_data[cosmetics_data['product_id'] == product_id].index[0]
    
    # Get the cosine similarity scores for the product
    cosine_scores = cosine_similarities2[product_index]
    # Sort the cosine similarity scores in descending order and get the top N similar products
    similar_product_indices = cosine_scores.argsort()[::-1][:N+1]
    similar_product_indices = similar_product_indices[similar_product_indices != product_index]
    # Filter out products with similarity score less than min_similarity
    similar_products = cosmetics_data.loc[similar_product_indices].copy()
    similar_products['similarity_score'] = cosine_scores[similar_product_indices]
    similar_products = similar_products[similar_products['similarity_score'] >= min_similarity]
    # Sort the similar products based on rating
    similar_products['rating'] = pd.to_numeric(similar_products['rating'], errors='coerce')
    similar_products.sort_values(by='rating', ascending=False, inplace=True)
    # Return the top N similar products
    return similar_products.iloc[:N]

# Example usage
get_top_similar_products2(322)

,product_name,category,ingredients,price,brand,rating,product_id,description,similarity_score
579,Peter Thomas Roth Peptide 21 Amino Acid Exfoli...,Peel,"['butylene glycol', 'sodium lactate', 'phytic ...",41.6,Peter Thomas Roth,5.00,894,Enjoy skin that looks smoother and more even w...,0.327278
355,Zelens PROVITAMIN D Fortifying Facial Mist 50ml,Mist,"['propanediol', 'glycerin', 'polyglyceryl-3 pc...",55.0,Zelens,5.00,670,Refresh and rehydrate thirsty skin with a few ...,0.306503
116,The Ordinary Buffet Supersize Serum 60ml,Serum,"['glycerin', 'lactococcus ferment lysate', 'ac...",24.4,The Ordinary,4.77,431,Rejuvenate your complexion with,0.519228
327,Zelens Z Balance Prebiotic and Probiotic Facia...,Mist,"['butylene glycol', 'sodium lactate', 'lactoba...",48.0,Zelens,4.60,642,Refresh your complexion with the,0.357348
0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",5.2,The Ordinary,4.55,315,Effectively hydrate skin with The Ordinary Nat...,0.588491
145,The Ordinary Amino Acids + B5,Serum,"['propanediol', 'betaine', 'sodium pca', 'pant...",5.9,The Ordinary,4.41,460,Hydrate and strengthen skin with,0.563906
683,Sarah Chapman Lash Boosting Eye Cleanse 70ml,Eye Care,"['glycerin', 'olea europaea fruit oil', 'rosa ...",33.0,Sarah Chapman Skinesis,2.00,998,Revitalise tired eyes and effortlessly remove ...,0.323411
367,ARK Skincare Hydrating Beauty Mist 150ml,Mist,"['glycerin', 'polysorbate-20', 'phenoxyethanol...",25.0,,0.00,682,,0.424353
2129,FAB Skin Lab Retinol Serum 0.25% Pure Concentrate,Treatment,"Water, Polysorbate 80, Caprylic/Capric Triglyc...",58.0,FIRST AID BEAUTY,0.00,2130,,0.312169
1253,FAB Skin Lab Retinol Serum 0.25% Pure Concentrate,Treatment,"['polysorbate', 'caprylic', 'capric', 'triglyc...",58.0,FIRST AID BEAUTY,0.00,115,,0.290561


### Getting user information

In [22]:
user = pd.read_csv('main.csv',encoding = 'latin_1')

In [23]:
user.head()

,Index,User_id,Product_id,Username,Stars,Review,Review_cleaned,Combination,Dry,Normal,Oily,Sensitive,Skin_Tone,Skin_Type,Eye_Color,Hair_Color,category,price
0,1,3420,157.0,allyp3,5,This is hands down the best cleanser I've ever...,"['hand', 'best', 'cleanser', 'ever', 'used', '...",0,0,0,0,0,Medium,Combination,Brown,Brunette,Cleanser,36.0
1,2,2483,157.0,PatTea,1,Unfortunately this doesn't work for everyone. ...,"['unfortunately', 'work', 'everyone', 'used', ...",0,0,0,0,0,Medium,Combination,Brown,Red,Cleanser,36.0
2,3,2715,157.0,Sabi1991,5,My favorite cleanser!! i love the packaging on...,"['favorite', 'cleanser', 'love', 'packaging', ...",0,0,0,0,0,No data,No data,No data,No data,Cleanser,36.0
3,4,4497,157.0,happyface2,5,I love all things Youth To The People! This cl...,"['love', 'thing', 'youth', 'people', 'cleanser...",0,0,0,0,0,Fair,Dry,Blue,Blonde,Cleanser,36.0
4,5,5017,157.0,kimkix34,5,I had a trial size of this and was obsessed. M...,"['trial', 'size', 'wa', 'obsessed', 'skin', 'f...",0,0,0,0,0,Fair,Normal,Green,Blonde,Cleanser,36.0


In [24]:
len(user.User_id.unique())

9837

In [25]:
len(user.Product_id.unique())

1075

In [26]:
user.rename(columns = {'User_id': 'user_id', 'Product_id': 'product_id'}, inplace = True)

In [29]:
user[user['user_id'] == 7005]

,Index,user_id,product_id,Username,Stars,Review,Review_cleaned,Combination,Dry,Normal,Oily,Sensitive,Skin_Tone,Skin_Type,Eye_Color,Hair_Color,category,price
8768,8769,7005,336.0,Sarah,5,This moisturiser is fantastic! I use it either...,NaN,0,0,0,0,0,No data,No data,No data,No data,Moisturiser,20.00
8784,8785,7005,333.0,Sarah,4,Very hydrating. I like that it is only a few i...,NaN,0,0,0,0,0,No data,No data,No data,No data,Moisturiser,30.00
8814,8815,7005,331.0,Sarah,5,I have very dry skin and have been struggling ...,NaN,0,1,0,0,0,No data,No data,No data,No data,Moisturiser,12.95
9386,9387,7005,431.0,Sarah,5,This serum is so worth the hype! My skin feels...,NaN,0,0,0,0,0,No data,No data,No data,No data,Serum,24.40
9724,9725,7005,491.0,Sarah,5,I have tried many skin renewal products and so...,NaN,0,0,0,0,0,No data,No data,No data,No data,Serum,60.00
9835,9836,7005,549.0,Sarah,5,Really pleased with purchase. Very moisturisin...,NaN,0,0,0,0,0,No data,No data,No data,No data,Oil,8.40
10087,10088,7005,598.0,sarah,5,Its price is very kind and it really helps me ...,NaN,0,0,0,0,0,No data,No data,No data,No data,Oil,32.00
10185,10186,7005,611.0,Sarah,5,"Love this product, left my skin feeling amazing!",NaN,0,0,0,0,0,No data,No data,No data,No data,Oil,20.99
10231,10232,7005,639.0,Sarah,5,This has such a lovely soft spray! It truly fe...,NaN,0,0,0,0,0,No data,No data,No data,No data,Mist,21.00
10270,10271,7005,642.0,Sarah,5,I can't get through the day without using mist...,NaN,0,0,0,0,0,No data,No data,No data,No data,Mist,48.00


#### Evaluation of Accuracy

In [30]:
def evaluate_accuracy(recommended_products, user_purchase_table):
    recommended_product_ids = set(recommended_products['product_id'])
    user_purchased_product_ids = set(user_purchase_table[user_purchase_table['user_id'] == user_id]['product_id'])
    overlap = recommended_product_ids.intersection(user_purchased_product_ids)
    return len(overlap) / len(recommended_product_ids) * 100

In [31]:
def evaluate_novelty(recommended_products, user_purchase_table):
    recommended_product_ids = set(recommended_products['product_id'])
    user_purchased_product_ids = set(user_purchase_table[user_purchase_table['user_id'] == user_id]['product_id'])
    novel_product_ids = recommended_product_ids - user_purchased_product_ids
    return len(novel_product_ids) / len(recommended_product_ids) * 100

In [32]:
def evaluate_coverage(recommended_products, user_purchase_table):
    recommended_product_ids = set(recommended_products['product_id'])
    user_purchased_product_ids = set(user_purchase_table[user_purchase_table['user_id'] == user_id]['product_id'])
    return len(recommended_product_ids) / len(user_purchased_product_ids) * 100

In [33]:
### Evluating Non-Personalised Recommendation

In [36]:
# Initialize lists to store evaluation metric values
accuracy_values = []
novelty_values = []
coverage_values = []

for user_id in user['user_id'].unique():
    # Get the user's purchase history
    user_history = user[user['user_id'] == user_id]
    
    # Iterate through each product in the user's purchase history
    for product_id in user_history['product_id'].unique():
        # Get the top similar products for the current product
        recommended_products = top_products_df
        
        # Evaluate the recommendation using the evaluation metrics
        accuracy = evaluate_accuracy(recommended_products, user_history)
        novelty = evaluate_novelty(recommended_products, user_history)
        coverage = evaluate_coverage(recommended_products, user_history)
        
        # Append the evaluation metric values to the lists
        accuracy_values.append(accuracy)
        novelty_values.append(novelty)
        coverage_values.append(coverage)

# Calculate the average evaluation metric values
average_accuracy = sum(accuracy_values) / len(accuracy_values)
average_novelty = sum(novelty_values) / len(novelty_values)
average_coverage = sum(coverage_values) / len(coverage_values)

print('Average accuracy:', average_accuracy)
print('Average novelty:', average_novelty)
print('Average coverage:', average_coverage)

Average accuracy: 0.06964125419072166
Average novelty: 99.93035874581153
Average coverage: 241202.41509433754


In [35]:
### Evaluation Personalised Recommendation

In [37]:
# Initialize lists to store evaluation metric values
accuracy_values = []
novelty_values = []
coverage_values = []

for user_id in user['user_id'].unique():
    # Get the user's purchase history
    user_history = user[user['user_id'] == user_id]
    
    # Iterate through each product in the user's purchase history
    for product_id in user_history['product_id'].unique():
        # Get the top similar products for the current product
        recommended_products = get_top_similar_products2(product_id)
        
        # Evaluate the recommendation using the evaluation metrics
        accuracy = evaluate_accuracy(recommended_products, user_history)
        novelty = evaluate_novelty(recommended_products, user_history)
        coverage = evaluate_coverage(recommended_products, user_history)
        
        # Append the evaluation metric values to the lists
        accuracy_values.append(accuracy)
        novelty_values.append(novelty)
        coverage_values.append(coverage)

# Calculate the average evaluation metric values
average_accuracy = sum(accuracy_values) / len(accuracy_values)
average_novelty = sum(novelty_values) / len(novelty_values)
average_coverage = sum(coverage_values) / len(coverage_values)

print('Average accuracy:', average_accuracy)
print('Average novelty:', average_novelty)
print('Average coverage:', average_coverage)

Average accuracy: 0.28679245283018867
Average novelty: 99.71320754716982
Average coverage: 824.9056603773596


In [39]:
# Initialize lists to store evaluation metric values
accuracy_values = []
novelty_values = []
coverage_values = []

for user_id in user['user_id'].unique():
    # Get the user's purchase history
    user_history = user[user['user_id'] == user_id]
    
    # Iterate through each product in the user's purchase history
    for product_id in user_history['product_id'].unique():
        # Get the top similar products for the current product
        recommended_products = get_top_similar_products1(product_id)
        
        # Evaluate the recommendation using the evaluation metrics
        accuracy = evaluate_accuracy(recommended_products, user_history)
        novelty = evaluate_novelty(recommended_products, user_history)
        coverage = evaluate_coverage(recommended_products, user_history)
        
        # Append the evaluation metric values to the lists
        accuracy_values.append(accuracy)
        novelty_values.append(novelty)
        coverage_values.append(coverage)

# Calculate the average evaluation metric values
average_accuracy = sum(accuracy_values) / len(accuracy_values)
average_novelty = sum(novelty_values) / len(novelty_values)
average_coverage = sum(coverage_values) / len(coverage_values)

print('Average accuracy:', average_accuracy)
print('Average novelty:', average_novelty)
print('Average coverage:', average_coverage)

Average accuracy: 0.23060796645702306
Average novelty: 99.76939203354297
Average coverage: 824.9056603773596


In [61]:
### To get the evaluation csv

In [62]:
user.head()

,User_id,Product_id,Username,Stars,Review,Review_cleaned,Combination,Dry,Normal,Oily,Sensitive,Skin_Tone,Skin_Type,Eye_Color,Hair_Color
0,3420,157.0,allyp3,5,This is hands down the best cleanser I've ever...,"['hand', 'best', 'cleanser', 'ever', 'used', '...",0,0,0,0,0,Medium,Combination,Brown,Brunette
1,2483,157.0,PatTea,1,Unfortunately this doesn't work for everyone. ...,"['unfortunately', 'work', 'everyone', 'used', ...",0,0,0,0,0,Medium,Combination,Brown,Red
2,2715,157.0,Sabi1991,5,My favorite cleanser!! i love the packaging on...,"['favorite', 'cleanser', 'love', 'packaging', ...",0,0,0,0,0,No data,No data,No data,No data
3,4497,157.0,happyface2,5,I love all things Youth To The People! This cl...,"['love', 'thing', 'youth', 'people', 'cleanser...",0,0,0,0,0,Fair,Dry,Blue,Blonde
4,5017,157.0,kimkix34,5,I had a trial size of this and was obsessed. M...,"['trial', 'size', 'wa', 'obsessed', 'skin', 'f...",0,0,0,0,0,Fair,Normal,Green,Blonde


In [40]:
df_test=pd.read_csv('test.csv')

In [41]:
df_test

,Index,User_id,Product_id,Username,Stars,Review,Review_cleaned,Combination,Dry,Normal,Oily,Sensitive,Skin_Tone,Skin_Type,Eye_Color,Hair_Color,category,price
0,7359,81,285.0,Abbey940,5,"I haven't personally used this product, but I ...","['personally', 'used', 'purchased', 'friend', ...",1,1,1,1,1,No data,No data,No data,No data,Moisturizer,80.00
1,3475,86,46.0,AbiRenee,5,I love this. It's clean and smells amazing. It...,"['love', 'clean', 'smell', 'amazing', 'super',...",1,1,1,1,1,Fair,Combination,Blue,Gray,Cleanser,34.00
2,540,88,310.0,AceofFace,5,The original remover was so awesome bring it b...,"['original', 'remover', 'wa', 'awesome', 'brin...",0,0,0,0,0,No data,No data,No data,No data,Cleanser,11.00
3,2541,90,87.0,AcidSunshine,5,I put this into my routine after loving the ni...,"['put', 'routine', 'loving', 'night', 'time', ...",1,1,1,1,1,Porcelain,Normal,Hazel,Brunette,Moisturizer,47.00
4,5232,104,109.0,Age,5,Highly recommended and cost-effective wipes. I...,"['highly', 'recommended', 'cost', 'effective',...",0,0,0,0,0,No data,No data,No data,No data,Cleanser,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,12438,9610,1171.0,fragmented.ME,4,For the past 30+ years all I've ever used on m...,NaN,0,0,0,0,0,No data,No data,No data,No data,Toner,7.95
579,12824,9750,1317.0,Crystal,4,Like the texture of the shower gel; very foamy...,NaN,0,0,0,0,0,No data,No data,No data,No data,Body Wash,8.50
580,12847,9776,1310.0,Saxon,5,I like the aroma of Bergamot but I find it har...,NaN,0,0,0,0,0,No data,No data,No data,No data,Body Wash,19.00
581,12520,9962,1213.0,Jules1806,5,"This has a really good consistency, drys brill...",NaN,0,1,0,1,0,No data,No data,No data,No data,Exfoliator,4.71


In [43]:

test_set_userids = set(df_test['User_id'].unique())

In [44]:
user_list = pd.DataFrame(test_set_userids,columns=['User_id'])
user_list

,User_id
0,8193
1,2051
2,8196
3,6154
4,6155
...,...
502,8172
503,4081
504,6134
505,8182


In [45]:
item_item_user_list = pd.DataFrame()
item_item_user_list = df_test[['User_id', 'Product_id']].copy()

In [46]:
item_item_user_list

,User_id,Product_id
0,81,285.0
1,86,46.0
2,88,310.0
3,90,87.0
4,104,109.0
...,...,...
578,9610,1171.0
579,9750,1317.0
580,9776,1310.0
581,9962,1213.0


In [47]:
item_item_user_list.to_csv('item_item_user_list.csv',index=False)

In [48]:
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['User_id', 'Product_id', 'Recommended_products'])

# Loop through the rows in test_data
for index, row in item_item_user_list.iterrows():
    user_id = row['User_id']
    product_id = row['Product_id']
    recommended_products = get_top_similar_products2(product_id)
    if recommended_products is not None:
        recommended_products = recommended_products['product_id'].tolist()
    else:
        recommended_products = []
    results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)

# Display the results
print(results_df)

/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Produ

/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Produ

/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Produ

/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Produ

     User_id  Product_id                               Recommended_products
0       81.0       285.0  [1023, 1498, 163, 1563, 1770, 1755, 164, 110, ...
1       86.0        46.0  [1833, 1499, 2307, 1836, 2126, 1991, 2248, 181...
2       88.0       310.0  [1952, 1909, 1954, 44, 1784, 244, 1899, 2275, ...
3       90.0        87.0  [1579, 172, 1465, 1563, 297, 246, 2452, 1703, ...
4      104.0       109.0  [2003, 67, 1873, 1880, 1911, 1921, 1946, 1800,...
..       ...         ...                                                ...
578   9610.0      1171.0  [1393, 1377, 1097, 1055, 1161, 1033, 1111, 690...
579   9750.0      1317.0  [1377, 1353, 1335, 1339, 722, 1336, 1113, 1352...
580   9776.0      1310.0  [1377, 1362, 690, 1892, 425, 2313, 1512, 2011,...
581   9962.0      1213.0  [1215, 1331, 1312, 1205, 1228, 375, 1344, 1389...
582  10048.0      1344.0  [1428, 1321, 1353, 1312, 463, 1308, 1422, 1279...

[583 rows x 3 columns]


/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Product_id': product_id, 'Recommended_products': recommended_products}, ignore_index=True)
/var/folders/n3/lq2fc8w11vscg7mbswjnpyj00000gn/T/ipykernel_60753/3283047144.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'User_id': user_id, 'Produ

In [49]:
results_df

,User_id,Product_id,Recommended_products
0,81.0,285.0,"[1023, 1498, 163, 1563, 1770, 1755, 164, 110, ..."
1,86.0,46.0,"[1833, 1499, 2307, 1836, 2126, 1991, 2248, 181..."
2,88.0,310.0,"[1952, 1909, 1954, 44, 1784, 244, 1899, 2275, ..."
3,90.0,87.0,"[1579, 172, 1465, 1563, 297, 246, 2452, 1703, ..."
4,104.0,109.0,"[2003, 67, 1873, 1880, 1911, 1921, 1946, 1800,..."
...,...,...,...
578,9610.0,1171.0,"[1393, 1377, 1097, 1055, 1161, 1033, 1111, 690..."
579,9750.0,1317.0,"[1377, 1353, 1335, 1339, 722, 1336, 1113, 1352..."
580,9776.0,1310.0,"[1377, 1362, 690, 1892, 425, 2313, 1512, 2011,..."
581,9962.0,1213.0,"[1215, 1331, 1312, 1205, 1228, 375, 1344, 1389..."


In [50]:
import itertools

# Group the results by user_id and aggregate the recommended_products lists
aggregated_results = results_df.groupby('User_id')['Recommended_products'].agg(lambda x: list(set(itertools.chain.from_iterable(x)))).reset_index()
aggregated_results.rename(columns={'Recommended_products': 'Recommended_products_list'}, inplace=True)

# Display the aggregated results
print(aggregated_results)

     User_id                          Recommended_products_list
0       81.0  [163, 164, 1770, 110, 1812, 1913, 1498, 1563, ...
1       86.0  [1819, 2307, 1991, 2248, 1833, 1836, 2126, 276...
2       88.0  [1952, 1954, 2275, 2616, 1899, 44, 244, 1909, ...
3       90.0  [1703, 297, 2569, 1579, 172, 1523, 2452, 246, ...
4      104.0  [1921, 67, 1860, 1800, 1873, 2003, 1973, 1911,...
..       ...                                                ...
502   9610.0  [1377, 1384, 1097, 1161, 1033, 1393, 690, 1111...
503   9750.0  [1377, 1378, 1352, 1353, 1416, 722, 1335, 1336...
504   9776.0  [1377, 1892, 1349, 1512, 425, 2313, 690, 1362,...
505   9962.0  [1312, 1344, 1228, 1389, 1331, 1205, 1397, 375...
506  10048.0  [1312, 1353, 1321, 332, 1422, 463, 1428, 1340,...

[507 rows x 2 columns]


In [51]:
aggregated_results_ordered = pd.merge(user_list,
                     aggregated_results,
                     on ='User_id',
                     how ='left')
aggregated_results_ordered

,User_id,Recommended_products_list
0,8193,"[1377, 1384, 1097, 1393, 690, 1300, 1398, 1306..."
1,2051,"[1664, 2179, 519, 1512, 617, 297, 1288, 219, 7..."
2,8196,"[1792, 1091, 1062, 71, 1033, 1038, 1774, 144, ..."
3,6154,"[163, 164, 1770, 1970, 2680, 1913, 1498, 1755,..."
4,6155,"[673, 1826, 388, 110, 1812, 1528, 24, 1468, 15..."
...,...,...
502,8172,"[1219, 1095, 1230, 1231, 942, 401, 703, 883, 9..."
503,4081,"[133, 2189, 1646, 2191, 147, 1684, 2164, 2103,..."
504,6134,"[2272, 1836, 1837, 2094, 1663, 1521, 49, 2101,..."
505,8182,"[2054, 923, 927, 936, 567, 955, 575, 2374, 109..."


In [52]:
aggregated_results_ordered.to_csv('item-item_result_2.csv', index = False)

In [76]:
#for i in range(0,506):
#    n = len(aggregated_results_ordered.Recommended_products_list[i])
#    if n > 11:
#        aggregated_results_ordered.Recommended_products_list[i] = aggregated_results_ordered.Recommended_products_list[i][:10]

In [77]:
#aggregated_results_ordered.to_csv('item-item_result_3.csv', index = False)

In [53]:
predictions_item_item = aggregated_results_ordered.Recommended_products_list

In [54]:
import math
def indicators_5(rankedList, testList):
    
    Hits_i = 0
    #Len_R = 0
    Len_T = len(testList) # maximum number of unique users
    #MRR_i = 0
    HR_i = 0
    HR_i2 = 0
    NDCG_i = 0    
    count = 0

    for i in range(len(testList)):
        for j in testList[i]:
            count = count + 1 #number of users+product combination
    
    for i in range(len(rankedList)):
        for j in range(len(rankedList[i])):
            for k in range(len(testList[i])):
                if testList[i][k]==rankedList[i][j]:
                    Hits_i+=1
                    HR_i+=1
                    HR_i2+=1
                    #MRR_i+=1/(j+1)   
                    NDCG_i+=1/(math.log2(1+j+1))
                    break
    HR_i/=Len_T #hit rate over total number of unique users
    HR_i2/=count #hit rate over total number of users+product combination
    if Hits_i == 0:
        NDCG_i = 0
    else:
        NDCG_i/=Hits_i #Dividing by Hits_i is a common way to normalize the NDCG score, and it can be useful when we don't know the ideal ranking (i.e., we don't know the IDCG). In this case, we assume that the ideal ranking contains all relevant items in the test set, and we normalize the NDCG score by the number of relevant items in the test set (i.e., Hits_i).
    print(Hits_i)
    print(f'HR_method1={HR_i}')
    print(f'HR_method2={HR_i2}')
    print(f'NDCG={NDCG_i}')

In [58]:
test_list = []
for i in test_set_userids:
    test_list.append(df_test[df_test['User_id'] == i]['Product_id'].tolist())

actual_test = pd.DataFrame(test_set_userids,columns=['user_id'])
actual_test['bought'] = test_list
actual_test

,user_id,bought
0,8193,[1362.0]
1,2051,[32.0]
2,8196,[1079.0]
3,6154,[233.0]
4,6155,[55.0]
...,...,...
502,8172,[1251.0]
503,4081,[73.0]
504,6134,[154.0]
505,8182,"[972.0, 1227.0]"


In [81]:
#if only limit to 10 earliest recommendation,result is not so good

#indicators_5(predictions_item_item,test_list)

In [59]:
indicators_5(predictions_item_item,test_list)

9
HR_method1=0.01775147928994083
HR_method2=0.015437392795883362
NDCG=0.3271790429835362
